## Import libraries

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [4]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F5.2 Zonal Statistics
#  Checkpoint:   F52c
#  Authors:      Sara Winsemius and Justin Braaten
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Copy properties to computed images

# Define a Landsat image.
img = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').first()

# Print its properties.
print('All image properties', img.propertyNames().getInfo())

# Subset the reflectance bands and unscale them.
computedImg = img.select('SR_B.').multiply(0.0000275).add(-0.2)

# Print the unscaled image's properties.
print('Lost original image properties', computedImg.propertyNames().getInfo())

# Subset the reflectance bands and unscale them, keeping selected
# source properties.
computedImg = img.select('SR_B.').multiply(0.0000275).add(-0.2) \
.copyProperties(img, ['system:time_start', 'LANDSAT_PRODUCT_ID'])

# Print the unscaled image's properties.
print('Selected image properties retained', computedImg \
.propertyNames().getInfo())

# Understanding which pixels are included in polygon statistics

# Define polygon geometry.
geometry = ee.Geometry.Polygon(
[
[
[-118.6019835717645, 37.079867782687884],
[-118.6019835717645, 37.07838698844939],
[-118.60036351751951, 37.07838698844939],
[-118.60036351751951, 37.079867782687884]
]
], None, False)

# Import the MERIT global elevation dataset.
elev = ee.Image('MERIT/DEM/v1_0_3')

# Define desired scale and crs for region reduction (for image display too).
proj = {
    'scale': 90,
    'crs': 'EPSG:5070'
}

# A count reducer will return how many pixel centers are overlapped by the
# polygon region.
count = elev.select(0).reduceRegion(
reducer = ee.Reducer.count(),
geometry = geometry,
scale = proj['scale'],
crs = proj['crs']
)
print('n pixels in the reduction', count.get('dem').getInfo())

# Make a feature collection of pixel center points for those that are
# included in the reduction.
pixels = ee.Image.pixelLonLat().reduceRegion(
reducer = ee.Reducer.toCollection(['lon', 'lat']),
geometry = geometry,
scale = proj['scale'],
crs = proj['crs']
)

def func_wjc(f):
    return f.setGeometry(ee.Geometry.Point([f.get('lon'), f \
    .get('lat')
    ]))

pixelsFc = ee.FeatureCollection(pixels.get('features')).map(
func_wjc)





# Display layers on the map.
Map.centerObject(geometry, 18)
Map.addLayer(
elev.reproject(
crs = proj['crs'],
scale = proj['scale']
),
{
    'min': 2500,
    'max': 3000,
    'palette': ['blue', 'white', 'red']
}, 'Image')
Map.addLayer(geometry, {
    'color': 'white'
}, 'Geometry')
Map.addLayer(pixelsFc, {
    'color': 'purple'
}, 'Pixels in reduction')



#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------





All image properties ['DATA_SOURCE_ELEVATION', 'WRS_TYPE', 'system:id', 'REFLECTANCE_ADD_BAND_1', 'REFLECTANCE_ADD_BAND_2', 'DATUM', 'REFLECTANCE_ADD_BAND_3', 'REFLECTANCE_ADD_BAND_4', 'REFLECTANCE_ADD_BAND_5', 'REFLECTANCE_ADD_BAND_6', 'REFLECTANCE_ADD_BAND_7', 'system:footprint', 'REFLECTIVE_SAMPLES', 'system:version', 'GROUND_CONTROL_POINTS_VERSION', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'UTM_ZONE', 'DATE_ACQUIRED', 'ELLIPSOID', 'system:time_end', 'DATA_SOURCE_PRESSURE', 'LANDSAT_PRODUCT_ID', 'STATION_ID', 'TEMPERATURE_ADD_BAND_ST_B10', 'DATA_SOURCE_REANALYSIS', 'REFLECTANCE_MULT_BAND_7', 'system:time_start', 'REFLECTANCE_MULT_BAND_6', 'L1_PROCESSING_LEVEL', 'PROCESSING_SOFTWARE_VERSION', 'L1_DATE_PRODUCT_GENERATED', 'ORIENTATION', 'REFLECTANCE_MULT_BAND_1', 'WRS_ROW', 'REFLECTANCE_MULT_BAND_3', 'REFLECTANCE_MULT_BAND_2', 'TARGET_WRS_ROW', 'REFLECTANCE_MULT_BAND_5', 'REFLECTANCE_MULT_BAND_4', 'THERMAL_LINES', 'TIRS_SSM_POSITION_STATUS', 'GRID_CELL_SIZE_THERMAL', 

## Display the interactive map

In [5]:
Map

Map(center=[37.079127385889684, -118.60117354470816], controls=(WidgetControl(options=['position', 'transparen…